# Trabalho Final Cocada
## Algoritmo de previsão de resultados de Partidas do Campeonato Brasileiro
### Aluno: João Victor Magno - 123711443

In [69]:
using LinearAlgebra
using Plots
using CSV
using DataFrames
using Dates
using MultivariateStats
using Statistics

### Abrindo o arquivo com os dados

In [70]:
partidas = CSV.read("campeonato-brasileiro-full.csv", DataFrame)
estatisticas = CSV.read("campeonato-brasileiro-estatisticas-full.csv", DataFrame)

Row,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios
,Int64,Int64,String15,Int64,Int64,String7?,Int64,String7?,Int64,Int64,Int64,Int64,Int64
1,1,1,Guarani,0,0,missing,0,missing,0,0,0,0,0
2,1,1,Vasco,0,0,missing,0,missing,0,0,0,0,0
3,2,1,Athletico-PR,0,0,missing,0,missing,0,0,0,0,0
4,2,1,Gremio,0,0,missing,0,missing,0,0,0,0,0
5,3,1,Flamengo,0,0,missing,0,missing,0,0,0,0,0
6,3,1,Coritiba,0,0,missing,0,missing,0,0,0,0,0
7,4,1,Goias,0,0,missing,0,missing,0,0,0,0,0
8,4,1,Paysandu,0,0,missing,0,missing,0,0,0,0,0
9,5,1,Internacional,0,0,missing,0,missing,0,0,0,0,0


### Juntando os dataframes

Junto as estatisticas com as partidas em um dataframe so, com as estatisticas dos mandantes e dos visitantes para facilitar o uso dos dados.

Mudo o tipo da coluna data de string para date e crio uma coluna nova ano, com o ano que aconteceu a partida.

In [71]:
rename_map_mandante = Dict(col => (col == "clube" ? "mandante" : "m_" * col) for col in names(estatisticas))

rename_map_visitante = Dict(col => (col == "clube" ? "visitante" : "v_" * col) for col in names(estatisticas))

estat_mandante = rename!(copy(estatisticas), rename_map_mandante)
estat_visitante = rename!(copy(estatisticas), rename_map_visitante)

j1 = leftjoin(partidas, estat_mandante, on = ["ID" => "m_partida_id", "mandante" => "mandante"])

j_final = leftjoin(j1, estat_visitante, on = ["ID" => "v_partida_id", "visitante" => "visitante"])

j_final.data = Date.(j_final.data, dateformat"dd/mm/yyyy")
j_final.ano = year.(j_final.data)
j_final

Row,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado,m_rodata,m_chutes,m_chutes_no_alvo,m_posse_de_bola,m_passes,m_precisao_passes,m_faltas,m_cartao_amarelo,m_cartao_vermelho,m_impedimentos,m_escanteios,v_rodata,v_chutes,v_chutes_no_alvo,v_posse_de_bola,v_passes,v_precisao_passes,v_faltas,v_cartao_amarelo,v_cartao_vermelho,v_impedimentos,v_escanteios,ano
,Int64,Int64,Date,Time,String15,String15,String15?,String15?,String31?,String31?,String15,String,Int64,Int64,String3,String3,Int64?,Int64?,Int64?,String7?,Int64?,String7?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,String7?,Int64?,String7?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64
1,1,1,2003-03-29,16:00:00,Guarani,Vasco,,missing,missing,missing,Guarani,Brinco de Ouro,4,2,SP,RJ,1,0,0,missing,0,missing,0,0,0,0,0,1,0,0,missing,0,missing,0,0,0,0,0,2003
2,2,1,2003-03-29,16:00:00,Athletico-PR,Gremio,,missing,missing,missing,Athletico-PR,Arena da Baixada,2,0,PR,RS,1,0,0,missing,0,missing,0,0,0,0,0,1,0,0,missing,0,missing,0,0,0,0,0,2003
3,3,1,2003-03-30,16:00:00,Flamengo,Coritiba,,missing,missing,missing,-,Maracanã,1,1,RJ,PR,1,0,0,missing,0,missing,0,0,0,0,0,1,0,0,missing,0,missing,0,0,0,0,0,2003
4,4,1,2003-03-30,16:00:00,Goias,Paysandu,,missing,missing,missing,-,Serra Dourada,2,2,GO,PA,1,0,0,missing,0,missing,0,0,0,0,0,1,0,0,missing,0,missing,0,0,0,0,0,2003
5,5,1,2003-03-30,16:00:00,Internacional,Ponte Preta,,missing,missing,missing,-,Beira Rio,1,1,RS,SP,1,0,0,missing,0,missing,0,0,0,0,0,1,0,0,missing,0,missing,0,0,0,0,0,2003
6,6,1,2003-03-30,16:00:00,Criciuma,Fluminense,,missing,missing,missing,Criciuma,Heriberto Hulse,2,0,SC,RJ,1,0,0,missing,0,missing,0,0,0,0,0,1,0,0,missing,0,missing,0,0,0,0,0,2003
7,7,1,2003-03-30,16:00:00,Juventude,Sao Paulo,,missing,missing,missing,-,Alfredo Jaconi,2,2,RS,SP,1,0,0,missing,0,missing,0,0,0,0,0,1,0,0,missing,0,missing,0,0,0,0,0,2003
8,8,1,2003-03-30,16:00:00,Fortaleza,Bahia,,missing,missing,missing,-,Castelão,0,0,CE,BA,1,0,0,missing,0,missing,0,0,0,0,0,1,0,0,missing,0,missing,0,0,0,0,0,2003
9,9,1,2003-03-30,16:00:00,Cruzeiro,Sao Caetano,,missing,missing,missing,-,Mineirão,2,2,MG,SP,1,0,0,missing,0,missing,0,0,0,0,0,1,0,0,missing,0,missing,0,0,0,0,0,2003


### Filtrando os dados

Como nem todas as partidas do banco tem as estatisticas corretas, eu filtrei para remover as linhas que tem estatisticas nulas do banco.

Separei dois dataframes das partidas, as que contem as colunas de precisão do passe e as que não tem.

Mudei também o tipo das colunas de precisão do passe e as de posse de bola de string para int.

In [72]:
replace_none_com_missing!(df) = replace!(df, "None" => missing)

banco = select!(j_final, Not([ :ID, :hora, :data, :rodata, :formacao_mandante, :formacao_visitante, :tecnico_mandante, :tecnico_visitante, :vencedor, :arena, :mandante_Estado, :visitante_Estado, :m_rodata, :v_rodata]))
replace_none_com_missing!(banco.m_posse_de_bola)
replace_none_com_missing!(banco.v_posse_de_bola)
replace_none_com_missing!(banco.m_precisao_passes)
replace_none_com_missing!(banco.v_precisao_passes)

partidas_sem_pp = banco[4987:end, :]
select!(partidas_sem_pp, Not([:m_precisao_passes, :v_precisao_passes]))
dropmissing!(partidas_sem_pp)
partidas_sem_pp.m_posse_de_bola = parse.(Int, replace.(partidas_sem_pp.m_posse_de_bola, "%" => ""))
partidas_sem_pp.v_posse_de_bola = parse.(Int, replace.(partidas_sem_pp.v_posse_de_bola, "%" => ""))

partidas_pp = banco[6126:end, :]
dropmissing!(partidas_pp)
partidas_pp.m_posse_de_bola = parse.(Int, replace.(partidas_pp.m_posse_de_bola, "%" => ""))
partidas_pp.v_posse_de_bola = parse.(Int, replace.(partidas_pp.v_posse_de_bola, "%" => ""))
partidas_pp.m_precisao_passes = parse.(Int, replace.(partidas_pp.m_precisao_passes, "%" => ""))
partidas_pp.v_precisao_passes = parse.(Int, replace.(partidas_pp.v_precisao_passes, "%" => ""))

1876-element Vector{Int64}:
 86
 72
 85
 85
 78
 82
 73
 77
 84
 81
  ⋮
 83
 76
 73
 79
 82
 82
 82
 82
 90

### Parâmetros para a busca

Aqui fiz buscas para acharmos a maquina de predicao de partidas entre time1 e time2, ou, partidas de algum determinado ano.

In [73]:
time1 = "Vasco"
time2 = "Botafogo-RJ"
ano = 2019

partidas_geral = partidas_sem_pp
partidas_1 = filter(row -> (row.mandante == time1 && row.visitante == time2) || (row.mandante == time2 && row.visitante == time1), partidas_sem_pp)

Row,mandante,visitante,mandante_Placar,visitante_Placar,m_chutes,m_chutes_no_alvo,m_posse_de_bola,m_passes,m_faltas,m_cartao_amarelo,m_cartao_vermelho,m_impedimentos,m_escanteios,v_chutes,v_chutes_no_alvo,v_posse_de_bola,v_passes,v_faltas,v_cartao_amarelo,v_cartao_vermelho,v_impedimentos,v_escanteios,ano
,String15,String15,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,Botafogo-RJ,Vasco,3,1,7,3,43,360,20,5,0,1,2,19,3,57,440,11,1,0,1,6,2017
2,Vasco,Botafogo-RJ,1,0,11,1,39,325,16,5,0,0,7,17,2,61,513,13,3,0,1,1,2017
3,Vasco,Botafogo-RJ,1,2,14,4,73,668,12,2,0,0,4,13,5,27,238,16,4,0,0,7,2018
4,Botafogo-RJ,Vasco,1,1,18,5,51,444,13,3,0,0,7,10,4,49,428,13,2,0,0,6,2018
5,Botafogo-RJ,Vasco,1,0,8,1,56,469,15,2,0,1,4,18,2,44,362,16,3,0,1,9,2019
6,Vasco,Botafogo-RJ,2,1,16,8,36,275,15,1,0,1,3,10,3,64,485,11,0,0,2,7,2019
7,Botafogo-RJ,Vasco,2,3,25,9,49,426,18,2,0,0,5,14,6,51,443,8,2,0,2,4,2020
8,Vasco,Botafogo-RJ,3,0,18,7,51,465,11,1,0,0,6,12,5,49,458,13,0,0,4,3,2021


### Achando os elementos do Mínimos Quadrados

Achamos o vetor $b$ realizando a subtração entre o número de gols do mandante e o número de gols do visitante, e, a matriz $A$ achamos retirando as colunas do dataframe que não precisamos e passando de dataframe para matriz. 

Usamos o método de resolução de mínimos quadrados para descobrimos a nossa maquina de predição.

- vetor_b = saldo de gols do mandante
- matriz_Jogos = matriz com as estatisticas da partida, cada linha é uma partida
- m_predicao = vetor com os pesos de influência no saldo de gols do mandante de cada estatística

In [74]:
vetor_b_1 = partidas_1.mandante_Placar .- partidas_1.visitante_Placar
vetor_b_geral = partidas_geral.mandante_Placar .- partidas_geral.visitante_Placar
matriz_A_1 = select!(partidas_1, Not([:mandante, :visitante, :ano, :mandante_Placar, :visitante_Placar]))
matriz_A_geral = select!(partidas_geral, Not([:mandante, :visitante, :ano, :mandante_Placar, :visitante_Placar]))
matriz_Jogos_1 = Matrix(matriz_A_1)
matriz_Jogos_geral = Matrix(matriz_A_geral)

m_predicao_1 = matriz_Jogos_1 \ vetor_b_1
m_predicao_geral = matriz_Jogos_geral \ vetor_b_geral

18-element Vector{Float64}:
 -0.007004774352456354
  0.20216586862360897
 -0.09674803642959635
  0.011294074242252289
  0.02268138314503014
 -0.11450076897301037
 -0.5444320858430263
  0.03648990652626924
 -0.025440233232255825
  0.012777782964193464
 -0.278961211102317
  0.10734454579124286
 -0.011146916728586387
 -0.022766335963570396
  0.023179290867403852
  0.39362333695348284
 -0.008769224664131284
  0.0489459271393042

### Partida média

Achamos uma partida média geral de todas as partidas do banco de dados e achamos uma partida do filtro 1.

In [75]:
media_jogos1 = mean(matriz_Jogos_1, dims=1)
media_jogosG = mean(matriz_Jogos_geral, dims=1)

matriz_jogos_comuns = [media_jogos1' media_jogosG']
matriz_jogos_comuns_t = matriz_jogos_comuns'

nomes_colunas = names(partidas_geral)
jComum_1 = DataFrame(matriz_jogos_comuns_t, nomes_colunas)

Row,m_chutes,m_chutes_no_alvo,m_posse_de_bola,m_passes,m_faltas,m_cartao_amarelo,m_cartao_vermelho,m_impedimentos,m_escanteios,v_chutes,v_chutes_no_alvo,v_posse_de_bola,v_passes,v_faltas,v_cartao_amarelo,v_cartao_vermelho,v_impedimentos,v_escanteios
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,14.625,4.75,49.75,429.0,15.0,2.625,0.0,0.375,4.75,14.125,3.75,50.25,420.875,12.625,1.875,0.0,1.375,5.375
2,14.5169,3.66324,51.7943,434.385,15.0003,2.16656,0.101194,1.70703,5.95023,11.134,2.76941,48.2057,408.339,15.0637,2.40312,0.152621,1.53782,4.51659


### Calculando a predição

Usamos a máquina de predição calculada anteriormente em um jogo para sabermos o saldo de gols do mandante.

Usei uma partida já existente para saber a confiabilidade da máquina e comparei o resultado obtido da previsão com o saldo real.

In [76]:
jogo_novo = matriz_Jogos_1[2, :]

saldo_mandante_1 = jogo_novo' * m_predicao_1
saldo_mandante_1_media = media_jogos1 * m_predicao_1

saldo_mandante_geral = jogo_novo' * m_predicao_geral
saldo_mandante_geral_media = media_jogosG * m_predicao_geral

println("Saldo de gols previsto (filtro 1): $(saldo_mandante_1)")
println("Saldo de gols previsto (geral): $(saldo_mandante_geral)")
println("Saldo de gols real: $(vetor_b_1[2])")
println()
println("Saldo de gols previsto da partida media (filtro 1): $(saldo_mandante_1_media)")
println("Saldo de gols previsto da partida media (geral): $(saldo_mandante_geral_media)")

Saldo de gols previsto (filtro 1): 1.0000000000000042
Saldo de gols previsto (geral): -0.06246818120741348
Saldo de gols real: 1

Saldo de gols previsto da partida media (filtro 1): [0.7499999999999966]
Saldo de gols previsto da partida media (geral): [0.45487723954874976]
